In [ ]:
# If you need to install libraries, use this:
!pip install pandas folium ortools numpy matplotlib

# Import libraries
import pandas as pd
import folium
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.pyplot as plt


In [2]:
# Sample data for depot and construction sites
data = {
    'location': ['Depot', 'Site 1', 'Site 2', 'Site 3', 'Site 4', 'Site 5'],
    'latitude': [40.7128, 40.7306, 40.7527, 40.7484, 40.7651, 40.7580],
    'longitude': [-74.0060, -73.9352, -73.9857, -73.9844, -73.9803, -73.9783],
    'demand': [0, 10, 15, 20, 25, 30]  # Material demand (in tons)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Show the data
df


,location,latitude,longitude,demand
0,Depot,40.7128,-74.0060,0
1,Site 1,40.7306,-73.9352,10
2,Site 2,40.7527,-73.9857,15
3,Site 3,40.7484,-73.9844,20
4,Site 4,40.7651,-73.9803,25
5,Site 5,40.7580,-73.9783,30


In [3]:
# Initialize a Folium map centered around the depot location
map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add the depot marker
folium.Marker([40.7128, -74.0060], popup='Depot', icon=folium.Icon(color='green')).add_to(map)

# Add markers for the construction sites
for index, row in df.iterrows():
    if row['location'] != 'Depot':
        folium.Marker([row['latitude'], row['longitude']], popup=f"{row['location']} - Demand: {row['demand']} tons").add_to(map)

# Display the map
map


In [4]:
# Haversine formula to calculate distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in kilometers
    dlon = np.radians(lon2 - lon1)
    dlat = np.radians(lat2 - lat1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Calculate the distance matrix
num_locations = len(df)
distance_matrix = np.zeros((num_locations, num_locations))

for i in range(num_locations):
    for j in range(num_locations):
        distance_matrix[i][j] = haversine(df['longitude'][i], df['latitude'][i], df['longitude'][j], df['latitude'][j])

# Display the distance matrix
distance_matrix


array([[0.        , 6.28626724, 4.7549752 , 4.35690781, 6.20550803,
        5.54146321],
       [6.28626724, 0.        , 4.91322801, 4.59343447, 5.39914436,
        4.7398705 ],
       [4.7549752 , 4.91322801, 0.        , 0.49051815, 1.45189486,
        0.85780212],
       [4.35690781, 4.59343447, 0.49051815, 0.        , 1.88879363,
        1.184698  ],
       [6.20550803, 5.39914436, 1.45189486, 1.88879363, 0.        ,
        0.80725389],
       [5.54146321, 4.7398705 , 0.85780212, 1.184698  , 0.80725389,
        0.        ]])

In [5]:
# Create data model for OR-Tools VRP solver
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['demands'] = df['demand'].tolist()
    data['vehicle_capacities'] = [50, 50]  # Two trucks, each with 50 tons capacity
    data['num_vehicles'] = 2
    data['depot'] = 0  # The depot is at index 0
    return data

# Solve VRP with capacity constraints
def solve_vrp():
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define cost of each arc (distance between locations)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add capacity constraint
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    # Set parameters for the search
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # Print the solution
    if solution:
        print_solution(data, manager, routing, solution)

# Print the solution
def print_solution(data, manager, routing, solution):
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            route.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route.append(manager.IndexToNode(index))
        print(f"Route for vehicle {vehicle_id}: {route} - Load: {route_load} tons")
        print(f"Distance of the route: {route_distance} km")
        total_distance += route_distance
    print(f"Total distance of all routes: {total_distance} km")

# Solve the VRP and display the results
solve_vrp()


Route for vehicle 0: [0, 2, 4, 1, 0] - Load: 50 tons
Distance of the route: 16 km
Route for vehicle 1: [0, 3, 5, 0] - Load: 50 tons
Distance of the route: 10 km
Total distance of all routes: 26 km
